## Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib as mpl
import torch
import os
import seaborn as sns

from utils.miscellaneous import get_pareto_front

In [ ]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

mpl.rcParams['figure.figsize'] = [7, 5]
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['savefig.bbox'] = 'tight'

mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['font.family'] = 'serif'

figures_folder = 'results'

# Plots

## Pareto-front

In [ ]:
df_MSGNN = pd.read_csv('results/Pareto_front/overview_MSGNN.csv')
df_GNN = pd.read_csv('results/Pareto_front/overview_GNN.csv')

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1.2]})

cmap = sns.cubehelix_palette(as_cmap=True)

# RMSE
axs[0].scatter(df_GNN['val_loss'], df_GNN[f'speed-up'], c=df_GNN['total parameters'], 
               norm=mpl.colors.LogNorm(vmin=1e4, vmax=1e6), cmap=cmap, marker='x')

axs[0].scatter(df_MSGNN['val_loss'], df_MSGNN[f'speed-up'], c=df_MSGNN['total parameters'], 
               norm=mpl.colors.LogNorm(vmin=1e4, vmax=1e6), cmap=cmap, marker='o')


pareto_front = get_pareto_front(df_GNN, 'val_loss', 'speed-up', ascending=True)
axs[0].plot(pareto_front[:,0], pareto_front[:,1], 'r--', linewidth=1, marker='')

pareto_front = get_pareto_front(df_MSGNN, 'val_loss', 'speed-up', ascending=True)
axs[0].plot(pareto_front[:,0], pareto_front[:,1], 'r--', linewidth=1, marker='')



# CSI
axs[1].scatter(df_GNN['val_CSI_005'], df_GNN[f'speed-up'], c=df_GNN['total parameters'], 
               norm=mpl.colors.LogNorm(vmin=1e4, vmax=1e6), cmap=cmap, marker='x')

scat = axs[1].scatter(df_MSGNN['val_CSI_005'], df_MSGNN[f'speed-up'], c=df_MSGNN['total parameters'], 
               norm=mpl.colors.LogNorm(vmin=1e4, vmax=1e6), cmap=cmap, marker='o')

pareto_front = get_pareto_front(df_GNN, 'speed-up', 'val_CSI_005', ascending=False)
axs[1].plot(pareto_front[:,1], pareto_front[:,0], 'r--', linewidth=1, marker='')

pareto_front = get_pareto_front(df_MSGNN, 'speed-up', 'val_CSI_005', ascending=False)
axs[1].plot(pareto_front[:,1], pareto_front[:,0], 'r--', linewidth=1, marker='')

clb = plt.colorbar(scat, ax=axs[1])
clb.set_label('N parameters')
clb.set_ticks([1e4, 1e5, 1e6])

axs[1].scatter(x=[], y=[], c='k', marker='o', label='mSWE-GNN')
axs[1].scatter(x=[], y=[], c='k', marker='x', label='SWE-GNN')
axs[0].set_xlabel('Val RMSE')
axs[0].set_ylabel('Speed-up')
axs[1].set_xlabel('Val CSI$_{0.05m}$')
axs[1].legend(title='Model', loc='upper left')
axs[0].spines[['right','top']].set_visible(False)
axs[1].spines[['right','top']].set_visible(False)
axs[0].set_ylim(0,1400)
axs[1].set_ylim(0,1400)
axs[1].set_xlim(0.45,0.9)
plt.tight_layout()

## Mass conservation loss

In [ ]:
df = pd.read_csv('results/mass_conservation.csv')
# df = df.drop(9)

from scipy import stats

fig, axs = plt.subplots(1,2, figsize=(13,5))

# loss
axs[0].scatter(df['trainer_options.conservation']*1e-6, df['val_loss'])
axs[0].set_xlabel(r'$\alpha_M$')
axs[0].set_ylabel('Validation loss')


# CSI
axs[1].scatter(df['trainer_options.conservation']*1e-6, df['val_CSI_005'])
axs[1].set_xlabel(r'$\alpha_M$')
axs[1].set_ylabel(r'Validation CSI$_{0.05}$')


axs[0].spines[['right','top']].set_visible(False)
axs[1].spines[['right','top']].set_visible(False)
axs[0].set_xscale('log')
axs[1].set_xscale('log')

# use scientific notation for y-axis
axs[0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

plt.tight_layout()

from scipy.stats import pearsonr

print('Loss:', pearsonr(df['trainer_options.conservation'], df['val_loss']))
print('CSI:', pearsonr(df['trainer_options.conservation'], df['val_CSI_005']))

## Batch size

In [ ]:
df = pd.read_csv('results/batch_prediction_times.csv')
grouped_df = df.groupby('num_parameters')

from utils.miscellaneous import get_numerical_times

numerical_times = get_numerical_times('multiscale_mesh_dataset_test', 
                                      20, 120, 48, time_start=0, time_stop=-1, 
                                      overview_file='database/overview.csv')

cmap=sns.cubehelix_palette(as_cmap=True)
norm = mpl.colors.LogNorm(vmin=4.8e4, vmax=4.8e5)
fig, ax = plt.subplots(figsize=(7,4))

for name, group in grouped_df:
    ax.plot(group['batch_size'], numerical_times.sum()/group['prediction_times'], '-', c=cmap(norm(group['num_parameters'].values[0])))
    ax.scatter(group['batch_size'], numerical_times.sum()/group['prediction_times'], s=3+2.5**group['K'].values[0], color=cmap(norm(group['num_parameters'].values[0])))

# add legend
for i in range(2,6):
    ax.scatter([], [], s=3+2.5**i, color='k', label=f'{i}')
ax.legend(title='GNN layers\n  per scale', fontsize='xx-small')

ax.set_xlabel('Batch Size')
ax.set_ylabel('Speed-up')

ax.set_xscale('log', base=2)
xlabel = [1, 2, 4, 8, 16]
ax.set_xticks(xlabel, xlabel)

ax.set_yscale('log', base=2)
ylabel = [50, 100, 200, 400, 800]
ax.set_yticks(ylabel, ylabel)

# add colorbar with discrete ticks
clb = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
clb.set_label('N parameters')
# clb.set_ticks([1e4, 1e5, 1e6])

# Average speed-up by batching (compare 20 to 1 batch size)
batching_speedup = (df.iloc[::6].prediction_times.values / df.iloc[5::6].prediction_times.values).mean()
print(f'Average speed-up by batching: {batching_speedup:.2f}')

## Dataset statistics

In [ ]:
from utils.load import load_dataset
from sklearn.model_selection import train_test_split

seed = 381
train_dataset = load_dataset('mesh_dataset', 80, seed, os.path.join('database/datasets', 'train'))
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.25, random_state=seed)
test_dataset = load_dataset('mesh_dataset', 20, seed=0, dataset_folder=os.path.join('database/datasets', 'test'))
test_dataset2 = load_dataset('dijkring_15_fine', 10, seed=0, dataset_folder=os.path.join('database/datasets', 'test'))

In [ ]:
print("DEM", "Num_cells", "Area", "Edge length", "Total volume")
dataset_names = ['Train', 'Validation', 'Test', 'Test2']
for i, dataset in enumerate([train_dataset, val_dataset, test_dataset, test_dataset2]):
    DEM = torch.cat([i.DEM for i in dataset])
    num_cells = np.array([i.num_nodes for i in dataset])
    area = torch.cat([i.area for i in dataset])
    edge_length = np.concatenate([i.mesh.edge_length for i in dataset])
    total_volume = np.array([(i.WD[:,-1]*i.area).sum() for i in dataset])/10e6

    print(dataset_names[i], '&',
          round(DEM.mean().item(), 2), '$\pm$', round(DEM.std().item(), 2), '&',
            round(num_cells.mean()), '$\pm$', round(num_cells.std()), '&',
            round(area.mean().item()), '$\pm$', round(area.std().item()), '&',
            round(edge_length.mean(), 1), '$\pm$', round(edge_length.std(), 1), '&',
            round(total_volume.mean().item(), 2), '$\pm$', round(total_volume.std().item(), 2), '\\\\')

### Hydrographs

In [ ]:
from utils.miscellaneous import plot_line_with_deviation

hydrograph_mesh_path = 'database/raw_datasets_mesh/Hydrograph'
hydrograph_dike_path = 'database/raw_datasets_dk15/Hydrograph'

train_hydrograph = np.array([np.loadtxt(os.path.join(hydrograph_mesh_path, f'Hydrograph_{i}.txt')) for i in range(1,80)])
test_hydrograph = np.array([np.loadtxt(os.path.join(hydrograph_mesh_path, f'Hydrograph_{i}.txt')) for i in range(81,100)])
dike_hydrograph = np.array([np.loadtxt(os.path.join(hydrograph_dike_path, f'Hydrograph_{i}.txt')) for i in range(101,112)])

time_vector = train_hydrograph[0][::2,0]/3600

fig, ax = plt.subplots(figsize=(7,5))

plot_line_with_deviation(time_vector, train_hydrograph[:,::2,1], with_minmax=True, ax=ax, label='Train')
plot_line_with_deviation(time_vector, test_hydrograph[:,::2,1], with_minmax=True, ax=ax, label='Test')
plot_line_with_deviation(time_vector, dike_hydrograph[:,::2,1], with_minmax=True, ax=ax, label='Test dike ring 15')

ax.set_xlabel('Time [h]')
ax.set_ylabel('Discharge [$m^3$/s]')
ax.legend(loc=5)